In [6]:
import numpy as np
from tqdm import tqdm
# what is tqdm?
import cv2
import os 
import shutil
import imutils
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

import plotly.graph_objs as ob
from plotly.offline import init_notebook_mode, iplot
from plotly import tools

from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras import layers
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import EarlyStopping

init_notebook_mode(connected=True)
RANDOM_SEED = 123

In [ ]:
# !where python

In [ ]:
# img_path = "C:/Users/racco/Documents/python_project/" \
# "braintumordetection/archive1/brain_tumor_dataset/"

# for kelas in os.listdir(img_path):
#     if not kelas.startswith("."):
#         img_num = len(os.listdir(img_path + kelas))
#         print(img_num)
#         print(os.listdir(img_path + kelas))
#         # img_num -> number of images are there 
#         for (n, file_name) in enumerate(os.listdir(img_path + kelas)):
#             img = img_path + kelas + "/" + file_name
#             if n < 5:
#                 shutil.copy(img, "TEST/" + kelas.upper() + "/" + file_name)
#             elif n < 0.8 * img_num:
#                 shutil.copy(img, "TRAIN/" + kelas.upper() + "/" + file_name)
#             else:
#                 shutil.copy(img, "VAL/" + kelas.upper() + "/" + file_name)

In [3]:
def load_data(dir_path, img_size=(100,100)):
    # purpose-> Load resized images as np.arrays to workspace
    X = []
    y = []
    i = 0
    labels = dict()
    for path in tqdm(sorted(os.listdir(dir_path))):
        if not path.startswith("."):
            labels[i] = path
            for file in os.listdir(dir_path + path):
                if not file.startswith("."):
                    img = cv2.imread(dir_path + path + "/" + file)
                    # img = cv2.resize(img, img_size)
                    X.append(img)
                    y.append(i)
            i += 1
    X = np.array(X)
    y = np.array(y)
    print(f"{len(X)} images loaded from {dir_path}")
    return X, y, labels

In [ ]:
def plot_confusion_matrix(cm, classes, normalize=False, title="Confusion Matrix", cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.figure(figsize=(6,6))
    plt.imshow(cm, interpolation="nearest", cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)
    if normalize:
        cm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix") #additional line for information
    threshold = cm.max() / 2.0
    cm = np.round(cm, 2)
    for i, j in intertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt .text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > threshold else "black")
    plt.tight_layout()
    plt.ylabel("True label")
    plt.xlabel("Predicted label")
    plt.show()

<div style="font-size: 13px;">

**Background about the Confusion Matrix**
<p> the function, plot_confusion_matrix, is used to plot 
the confusion matrix for a classification model.
It is used to evaluate the performance of a classification model by
showing the true positives, false positives, true negatics and false negatives for each class.
</p>

1. **Function Definition**
    ```python
    def plot_confusion_matrix(cm, classes,
                              normalize=False,
                              title='Confusion matrix',
                              cmap=plt.cm.Blues):
    ```

    - **`cm`** → This is the confusion matrix itself, a 2D numpy array representing the number of correct and incorrect predictions made by the model for each class. (Define class, in this setting).
    - **`classes`** → A list of class names that the confusion matrix represents (like e.g. `["Class 1", "Class 2"]`).
    - **`normalize`** → A boolean flag that indicates whether to normalize the confusion matrix (that is, divide the values by the total sum of each row, so that the values are proportions rather than counts = basic linear algebra).
    - **`title`** -> a string represnting the title of the plot
    - **`cmap`** -> color map for the confusion matric (the default is as written in the code) = it is color gradient
    - **``**
    - **``**

3. **Plotting the Confusion Matrix**
    - Sub-item 1
    - Sub-item 2

4. **Main item**
    - Sub-item 1
    - Sub-item 2

5. **Main item**
    - Sub-item 1
    - Sub-item 2



</div>




In [11]:
train_dir = "C:/Users/racco/Documents/python_project/braintumordetection/TRAIN/"
test_dir = "C:/Users/racco/Documents/python_project/braintumordetection/TEST/"
val_dir = "C:/Users/racco/Documents/python_project/braintumordetection/VAL/"
img_size = (224, 224)

# use predefined function to load the image data into workspace
X_train, y_train, labels_train = load_data(train_dir, img_size)
X_test, y_test, _ = load_data(test_dir, img_size)
X_val, y_val, _ = load_data(val_dir, img_size)

# what is the purpose of "load_data" function?

100%|██████████| 2/2 [00:00<00:00, 10.53it/s]


193 images loaded from C:/Users/racco/Documents/python_project/braintumordetection/TRAIN/


100%|██████████| 2/2 [00:00<00:00, 266.40it/s]


10 images loaded from C:/Users/racco/Documents/python_project/braintumordetection/TEST/


100%|██████████| 2/2 [00:00<00:00, 34.12it/s]

50 images loaded from C:/Users/racco/Documents/python_project/braintumordetection/VAL/
